In [53]:
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
import torchvision
import os
import shutil

In [54]:
base_dir = r"D:\CODE\jupyter notebook\data\weather_data\4weather"
specises = ['cloudy', 'rain', 'shine', 'sunrise']

In [55]:
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
if not os.path.isdir(base_dir):
    os.mkdir(base_dir)
    os.mkdir(train_dir)
    os.mkdir(test_dir)
    for train_or_test in ['train', 'test']:
        for specy in specises:
            os.mkdir(os.path.join(base_dir, train_or_test, specy))

In [56]:
img_dir = r"D:\CODE\jupyter notebook\data\weather_data\weather_data"
for i, img in enumerate(os.listdir(img_dir)):
    if i%5 == 0:
        mode = "test"
    else:
        mode = "train"
    for spec in specises:
        if spec in img:
            s = os.path.join(img_dir, img)
            d = os.path.join(base_dir, mode, spec, img)
            shutil.copy(s, d)

In [57]:
for train_or_test in ['train', 'test']:
    for spec in specises:
        print(train_or_test, spec, len(os.listdir(os.path.join(base_dir, train_or_test, spec))))

train cloudy 240
train rain 172
train shine 202
train sunrise 286
test cloudy 60
test rain 43
test shine 51
test sunrise 71


In [58]:
from torchvision import transforms
from torch.utils.data import DataLoader

In [59]:
transforms = transforms.Compose([
    transforms.Resize((30, 30)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [60]:
train_dataset = torchvision.datasets.ImageFolder(train_dir, transforms)
test_dataset = torchvision.datasets.ImageFolder(test_dir, transforms)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [61]:
imgs, labels = next(iter(train_dataloader))
imgs.size()

torch.Size([64, 3, 30, 30])

In [62]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = nn.Conv2d(3, 18, kernel_size=5, padding=2)
        self.conv_2 = nn.Conv2d(18, 36, kernel_size=5)
        self.avgpool_1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.avgpool_2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.sigmoid = nn.Sigmoid()
        self.linear_1 = nn.Linear(900, 128)
        self.linear_2 = nn.Linear(128, 64)
        self.linear_3 = nn.Linear(64, 4)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
    def forward(self, input):
        x = self.conv_1(input)
        x = self.sigmoid(x)
        x = self.avgpool_1(x)
        x = self.conv_2(x)
        x = self.sigmoid(x)
        x = self.avgpool_2(x)
        x = self.flatten(x)
        x = x.view(-1, 900)
        x = self.linear_1(x)
        x = self.relu(x)
        x = self.linear_2(x)
        x = self.relu(x)
        output = self.linear_3(x)
        return output

In [63]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
model = Model().to(device)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01)

In [64]:
def train(dataloader, model, loss_fn, opt, device):
    train_loss, train_correct = 0, 0
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        with torch.no_grad():
            train_loss += loss
            train_correct += (y_pred.argmax(1)==y).type(torch.int32)
    train_loss /= num_batch
    train_correct /= size
    return train_correct, train_loss

def test(dataloader, model, loss_fn, device):
    test_loss, test_correct = 0, 0
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        test_loss += loss
        test_correct += (y_pred.argmax(1)==y).type(torch.int32)
    test_loss /= num_batch
    test_correct /= size
    return test_correct, test_loss

In [65]:
epoches = 100
train_loss = []
train_correct = []
test_loss = []
test_correct = []
for epoch in range(epoches):
    epoch_train_correct, epoch_train_loss = train(train_dataloader, model, loss_fn, opt, device)
    epoch_test_correct, epoch_test_loss = test(test_dataloader, model, loss_fn, device)
    train_loss.append(epoch_train_loss)
    train_correct.append(epoch_train_correct)
    test_loss.append(epoch_test_loss)
    test_correct.append(epoch_test_correct)
    template = ("epoch:{:2d}, train_loss:{:.5f}, train_correct:{:.2f}%, test_loss:{:.5f}, test_correct:{:.2f}%")
    print(template.format(epoch, epoch_train_loss, epoch_train_correct*100, epoch_test_loss, epoch_test_correct*100))

RuntimeError: The size of tensor a (64) must match the size of tensor b (4) at non-singleton dimension 0